In [22]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

import os
print(os.listdir("../input"))

['train.csv', 'sample_submission.csv', 'test.csv']


In [83]:
main_train = pd.read_csv("../input/train.csv")
main_test = pd.read_csv("../input/test.csv")

In [84]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Create a train/test validation split
train, test = train_test_split(main_train, test_size=0.2)

# Seperate output and input from DataFrame
train_output = to_categorical(train['label'])
test_output = to_categorical(test['label'])

train_input_raw = train.drop('label', axis=1)
test_input_raw = test.drop('label', axis=1)

def resize1DArray(in_array):
    out_array = []
    for index, row in in_array.iterrows():
        out_row = row.values.reshape(28,28)
        out_array.append(out_row)
        
    return(out_array)

# Train & test arrays
train_input = resize1DArray(train_input_raw)
test_input = resize1DArray(test_input_raw)
test_final = resize1DArray(main_test)

# Convert to numpy arrays
train_input = np.array(train_input).reshape(-1, 28, 28, 1)
test_input = np.array(test_input).reshape(-1, 28, 28, 1)
test_final = np.array(test_final).reshape(-1, 28, 28, 1)


In [86]:
print(test_input.shape)

(8400, 28, 28, 1)


In [87]:
model = Sequential()

# Input 28x28 matrecies representing images at grayscale
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [88]:
# Train and test model
model.fit(train_input, train_output, validation_data=(test_input, test_output), epochs=3)

Train on 33600 samples, validate on 8400 samples
Epoch 1/3
33600/33600 [==============================] - 76s 2ms/step - loss: 0.6172 - acc: 0.9233 - val_loss: 0.1167 - val_acc: 0.9665
Epoch 2/3
33600/33600 [==============================] - 76s 2ms/step - loss: 0.0697 - acc: 0.9784 - val_loss: 0.1290 - val_acc: 0.9654
Epoch 3/3
33600/33600 [==============================] - 76s 2ms/step - loss: 0.0414 - acc: 0.9877 - val_loss: 0.1374 - val_acc: 0.9705


In [89]:
results = model.predict(test_final)

In [110]:
results_nums = []
for result in results:
    largestIndex = np.argmax(result)
    results_nums.append(largestIndex)
    
ids = range(1, len(results_nums) + 1)
result_dataframe = pd.DataFrame({'ImageId': ids, 'Label': results_nums})

In [112]:
result_dataframe.to_csv('submission.csv', index=False)